In [1]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

In [2]:
import torch
import torch.nn as nn

In [19]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

# device = torch.device('cpu')
device,torch.backends.mps.is_built(),torch.backends.mps.is_available(),torch.cuda.is_available()

(device(type='cuda'), False, False, True)

In [4]:
from datasets import load_dataset
# dataset = load_dataset("wmt14","de-en")
dataset2 = load_dataset("wmt14","fr-en")

Using the latest cached version of the module from /home/ubuntu/.cache/huggingface/modules/datasets_modules/datasets/wmt14/d5cfc45c32d826941d8678bf74c810c2aaa057cdc5544f1e23a5dab8c0407a9f (last modified on Sun Mar 31 14:29:52 2024) since it couldn't be found locally at wmt14, or remotely on the Hugging Face Hub.


Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [23]:
src_vocab_size = tokenizer.vocab_size
tgt_vocab_size = tokenizer.vocab_size
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 512
dropout = 0.1
batch_size = 64

transformer_model = nn.Transformer().to(device)

/media/ubuntu/7ACA674ACA67022D/void-diffusion/env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))

In [8]:
transformer_model(src, tgt).shape


torch.Size([20, 32, 512])

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") # split + toInt
tokenizer.pad_token = tokenizer.eos_token
tokenizer.vocab_size

50257

In [25]:
def tokenization(batch):
    translation = batch['translation']
    source = [sample['en'] for sample in translation]
    target = [sample['fr'] for sample in translation]
    source_tokenized = tokenizer(source, padding='max_length',max_length=max_seq_length)
    target_tokenized = tokenizer(target, padding='max_length',max_length=max_seq_length)
    
    source_tensor = torch.tensor(source_tokenized['input_ids'], dtype=torch.float).to(device)
    target_tensor = torch.tensor(target_tokenized['input_ids'], dtype=torch.float).to(device)
    
    return source_tensor, target_tensor

In [26]:
for i, batch in enumerate(dataset2['train'].iter(batch_size)):
    src,tgt = tokenization(batch)
    print(src,tgt,transformer_model(src, tgt).shape)
    break

tensor([[4.9650e+03, 2.4098e+04, 2.8600e+02,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        [4.0000e+01, 1.3627e+04, 2.8291e+04,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        [7.0030e+03, 1.1000e+01, 3.5500e+02,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        ...,
        [1.7220e+03, 6.6100e+02, 4.2300e+02,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        [1.8580e+03, 3.1800e+02, 1.1000e+01,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        [1.1350e+03, 4.6600e+02, 4.0700e+02,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04]], device='cuda:0') tensor([[6.2070e+03, 1.7163e+04, 3.9000e+02,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        [4.0932e+04, 3.9073e+04, 5.6500e+02,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        [5.3770e+03, 1.3260e+03, 4.1000e+02,  ..., 5.0256e+04, 5.0256e+04,
         5.0256e+04],
        ...,
        [3.5000e+01, 6.0000e+00, 1.4272e+04,  ..., 5.0256e+04, 5.0256e+04,
        

/media/ubuntu/7ACA674ACA67022D/void-diffusion/env/lib/python3.11/site-packages/torch/nn/functional.py:5527: UserWarning: 1Torch was not compiled with memory efficient attention. (Triggered internally at ../aten/src/ATen/native/transformers/hip/sdp_utils.cpp:505.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


In [27]:
from torchtext.data.metrics import bleu_score

https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/more_advanced/seq2seq_transformer/seq2seq_transformer.py
https://github.com/gordicaleksa/pytorch-original-transformer
https://zhuanlan.zhihu.com/p/581334630